In [1]:
from imports import *
from functions import *
from auth import *
from constants import *

In [ ]:
driver = create_driver()


In [52]:
wait = WebDriverWait(driver, 3)

In [54]:
login(driver, email, password)

⏳ Tentative de suppression du popup Fincaraiz...
🧨 Pop-up Fincaraiz supprimé.
Erreur lors de la connexion :
name 'wait' is not defined


Type of contract

In [ ]:
buy_rent_projects ="arriendo"
choose_rent_buy_projects(driver, buy_rent_projects)

Option trouvée : Venta
>> 'Venta' sélectionné.


Type of Real Estate

In [ ]:
real_estate_types = ["apartamento", "apartaestudio"]

select_real_estate_types(driver, real_estate_types)

✅ Sélections supprimées.
